In [23]:
import mplhep
import numpy as np
import pandas as pd
import uproot
from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import auc, roc_curve
from sklearn.model_selection import KFold
from xgboost import XGBClassifier

In [25]:
import import_ipynb
import First_look_at_data.ipynb

ModuleNotFoundError: No module named 'First_look_at_data.ipynb'; 'First_look_at_data' is not a package